In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from dython.nominal import associations

from sklearn.metrics import roc_curve
from sklearn.metrics import balanced_accuracy_score

# Ignore future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Load Data

In [ ]:
IDS_MAPPING_FN = "./data/IDS_mapping.csv"
DIABETIC_FN = "./data/diabetic_data.csv"

In [ ]:
# read files
mapping = pd.read_csv(IDS_MAPPING_FN, header=None)
df = pd.read_csv(DIABETIC_FN)

In [ ]:
MEDIC_COLUMNS = df.columns[24:46].tolist()
MEDIC_COLUMNS_TAKE = ["take_" + med for med in MEDIC_COLUMNS]
PREVIOUS_HOSPITAL_ENCOUNTERS = ["number_outpatient", "number_inpatient", "number_emergency"]

Create dictionary of code-value mappings of `admission_type_id`, `discharge_disposition_id`, and `admission_source_id` using the mapping provided in the data-folder, and map integer values to string values for readability. 

In [ ]:
admission_type_dict = {}
discharge_disposition_dict = {}
admission_source_dict = {}

list1 = []
for i, j in zip(mapping[0].values, mapping[1].values):
  if len(str(i))>3:
    feature_name = i
  elif len(str(i))!=3:
    if feature_name == 'admission_type_id':
      admission_type_dict[int(i)] = j
    elif feature_name == 'discharge_disposition_id':
      discharge_disposition_dict[int(i)] = j
    elif feature_name == 'admission_source_id':
      admission_source_dict[int(i)] = j


df['admission_type'] = df['admission_type_id'].apply(lambda x: admission_type_dict[x])
df['discharge_disposition'] = df['discharge_disposition_id'].apply(lambda x: discharge_disposition_dict[x])
df['admission_source'] = df['admission_source_id'].apply(lambda x: admission_source_dict[x])

In [ ]:
print(f"Number of unique encounters: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")

## Preprocessing

Some patients have many encounters (up to 40).

We only keep the first observation for each unique patient to treat them as i.i.d random variables.

We filter to only keep observations with `admission_type` $\in$ [Emergency, Urgent, Elective].

In [ ]:
df = df.groupby("patient_nbr").first(skipna=False).reset_index()
df = df[df['admission_type'].isin(['Emergency', 'Urgent', 'Elective'])]
print(f"Number of unique encounters after only keeping first encounter for each patient and filtering by admission_type : {df.shape[0]}")

#### Feature Engineering: Diabetic Information

To select features as predictor variables we are only interested in certain information about the patients, but not the exact values. Thus we aggregate the information of sets of columns into new columns:


* Blood Glucose Tests

    * `max_glu_serum_flag`: Whether a max glucose serum test was done at the hospital : `max_glu_serum`


    * `A1C_flag`: Whether an AC1 test was performed to monitor blood glucose levels : `AC1result`

    
* Diabetic Medication Information: 

    * `change_dosage`: Whether there was any change ("Up" or "Down") in the diabetic prescription dosages as a result of the hospital visit.
        * *The column `change` indicates if there was a change in diabetic medications (either dosage or generic name). So if `change` is marked as changed but `change_dosage` is none, then there must have been a change in the generic name, i.e. the chemical name of a medicine.*


    * `change_medicine`: Whether there was prescribed any new diabetic medication i.e. a change in the medicament as a result of the hospital visit.


    * `num_diabetic_prescriptions`: How many diabetic prescriptions the patient ongoingly had at the time of hospital visit - a count of entries that are $\in \{\text{"Steady"}, \text{"Up"}, \text{"Down"}\}$ in `MEDIC_COLUMNS`

    * `take_<medicine_name>`: Whether the patient takes the <medicine_name> 


* Admitted in the hospital within the previous year

    * `prev_year_hospital`: Whether the patient had any admissions in the hospital during the past year. 

* Health Insurance / Coverage

    * `blue_cross`: Patient has private insurance
    * `medicaid`: Patient has medicaid
    * `medicare`: Patient has medicare
    * `self_payed`: Patient payed up front
  
* `discharge_disposition_id`: is aggregated into `home`, `transfer`, `unknown`, and `other`

* `Age` is aggregated into `[0-30)`, `[30-60)`, and `[60-100)`

* Readmitted
    * `readmitted_flag`: Whether the patient was readmitted or not based on the columns `readmitted`


In [ ]:
#### Blood glucose Tests ####
df['max_glu_serum_flag'] = df['max_glu_serum'].notnull().astype(int) # Max Glucose Serum test flag
df['A1C_flag'] = df['A1Cresult'].notnull().astype(int) # AC1 test flag

#### Diabetic Medication Information ####
# Change in dosage if any of the diabetics prescriptions has entries "Up" or "Down"
df['change_dosage'] = df[MEDIC_COLUMNS].isin(['Up', 'Down']).any(axis=1).astype(int)
# Check if each entry is in the set ['Up', 'Down', 'Steady'] and sum all True entries for each row
df['num_diabetic_prescriptions'] = df[MEDIC_COLUMNS].apply(lambda col: np.isin(col, ['Up', 'Down', 'Steady'])).sum(axis=1).astype(int)
# Change in medicine is assumed to be the case when original change column = 1 but change in dosage = 0
df['change_medicine'] = np.where((df['change'] == 'Ch') & (df['change_dosage'] == 0), 1, 0)

# Whether the patient takes any of the medications
for med in MEDIC_COLUMNS:
    name = "take_" + med
    df[name] = df[med].isin(["Down", "Steady", "Up"])

#### Hospital Encounters during preceding year ####
df['prev_year_hospital'] = (df[PREVIOUS_HOSPITAL_ENCOUNTERS] > 0).any(axis=1).astype(int)

#### Insurance Billing ####
df['blue_cross'] = np.where(df["payer_code"]=="BC", 1, 0)
df['medicare'] = np.where(df["payer_code"]=="MC", 1, 0)
df['medicaid'] = np.where(df["payer_code"]=="MD", 1, 0)
df['self_pay'] = np.where(df["payer_code"]=="SP", 1, 0)

#### Discharge disposition id aggregated ####
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace([1, 6, 8, 13], "home")
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace([2, 3, 4, 5, 9, 10, 14, 15, 16, 17, 22, 23, 24, 27, 28, 29], "transfer")
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace([18, 25, 26], "unknown")
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace([7, 11, 12, 19, 20, 21], "other")

#### Readmitted ####
# y_i
df['readmitted_flag'] = np.where(df['readmitted']=='NO', 0, 1) # Readmitted flag

#### Age aggregaed ####
df['age'] = df['age'].replace(['[0-10)', '[10-20)','[20-30)'], '[0-30)')
df['age'] = df['age'].replace(['[30-40)', '[40-50)','[50-60)'], '[30-60)')
df['age'] = df['age'].replace(['[60-70)', '[70-80)','[80-90)', '[90-100)'], '[60-100)')

## Define variables

In [ ]:
FEATURES_NUM = ["time_in_hospital", "num_lab_procedures", "num_procedures", 'num_diabetic_prescriptions', 
                "num_medications", 'prev_year_hospital', "number_diagnoses"]

FEATURES_BIN = ['max_glu_serum_flag', 'A1C_flag', 'change_dosage', 'change_medicine', "blue_cross", "medicaid", "medicare","self_pay"]

FEATURES_CAT = ['discharge_disposition_id', "admission_type_id"]

FEATURES = FEATURES_NUM + FEATURES_BIN + FEATURES_CAT + MEDIC_COLUMNS_TAKE

PROTECTED_FEATURES = ['age','race', 'gender']
PATIENTS = ["patient_nbr"]  
TARGET = ["readmitted_flag"]

In [ ]:
df = df[PATIENTS + FEATURES + PROTECTED_FEATURES + TARGET]

In [ ]:
df_one_hot = pd.get_dummies(df, prefix=None, prefix_sep='_', dummy_na=False, columns=FEATURES_CAT, drop_first=False) # dropf irst?
df_one_hot = df_one_hot.drop(["discharge_disposition_id_unknown"], axis = 1)
FEATURES_ONE_HOT = df_one_hot.drop(PATIENTS+PROTECTED_FEATURES+TARGET, axis=1).columns.tolist()

Undersample majority class

In [ ]:
df_major = df_one_hot[df_one_hot['readmitted_flag'] == 0]
df_minor = df_one_hot[df_one_hot['readmitted_flag'] == 1]

df_downsamples = resample(df_major, replace=False, n_samples=len(df_minor), random_state=42)  

df_one_hot = pd.concat([df_downsamples, df_minor])

Define train/test split

In [ ]:
X = df_one_hot[FEATURES_ONE_HOT]
y = df_one_hot[TARGET].to_numpy().reshape(-1)
protected_features = df_one_hot[PROTECTED_FEATURES]

X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(X, y, protected_features, test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train[FEATURES_NUM])
X_train[FEATURES_NUM] = scaler.transform(X_train[FEATURES_NUM])
X_test[FEATURES_NUM] = scaler.transform(X_test[FEATURES_NUM])

In [ ]:
# corr_matrix = associations(pd.concat([df_one_hot[FEATURES_ONE_HOT], df_one_hot[TARGET]], axis=1), figsize=(18, 15))


# plt.figure(figsize=(12, 10))
# sns.heatmap(corr_matrix, annot=True, cmap='vlag', fmt=".2f")
# plt.title("Correlation Heatmap of Features with Target")
# plt.show()

Using Grid search and Cross Validation we found an inverse regularisation strength of $0.1$  

In [ ]:
logistic_model = LogisticRegression(max_iter=5000, penalty='l2', C=0.1, tol=1e-4, solver = "saga")
logistic_model.fit(X_train, y_train)
y_pred = logistic_model.predict(X_test)
y_pred_proba_0 = logistic_model.predict_proba(X_test)[:,0]
y_pred_proba_1 = logistic_model.predict_proba(X_test)[:,1]

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
print("Accuracy:", accuracy)
print("False Positive Rate:", fpr[1])
print("True Positive Rate:", tpr[1])

In [ ]:
df_protected_features_predicions =  pd.concat([group_test, pd.DataFrame({'y_test': y_test, 'y_pred': y_pred, "y_proba_0":y_pred_proba_0, "y_proba_1":y_pred_proba_1}, index=X_test.index)], axis=1)

In [ ]:
def get_metrics(group_name, df_protected_features_predicions=df_protected_features_predicions):
    group_names, accuracies, tprs, fprs = [], [], [], []

    # Group by 'age', 'race', and 'gender' columns
    groups = df_protected_features_predicions.groupby(group_name)

    for group, group_df in groups:
        group_names.append(group)
        y_true = group_df['y_test']
        y_pred = group_df['y_pred']
        
        accuracy = accuracy_score(y_true, y_pred)
        accuracies.append(accuracy)
        
        fpr, tpr, thresholds = roc_curve(y_true, y_pred)
        tprs.append(tpr[1])
        fprs.append(fpr[1])
        
    return group_names, accuracies, tprs, fprs

In [ ]:
group_names, accuracies, tprs, fprs = get_metrics('age')

In [ ]:
print(group_names, tprs, fprs)

In [ ]:
def y_proba_protected_groups(group, df_protected_features_predicions=df_protected_features_predicions):
    proba_preds, true, protected_groups = [], [], []

    for group, group_df in df_protected_features_predicions.groupby(group):
        y_true = group_df['y_test']
        y_proba_0 = group_df["y_proba_0"]
        y_proba_1 = group_df["y_proba_1"]
        protected_groups.append(group)
        proba_preds.append(np.column_stack((y_proba_0,y_proba_1)))
        true.append(y_true)
        
    return proba_preds, true, protected_groups

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def calculate_tpr_fpr(y_real, y_pred):
    # Calculates the confusion matrix and recover each element
    cm = confusion_matrix(y_real, y_pred)
    TN = cm[0, 0]
    FP = cm[0, 1]
    FN = cm[1, 0]
    TP = cm[1, 1]
    # Calculates tpr and fpr
    tpr =  TP/(TP + FN) # sensitivity - true positive rate
    fpr = 1 - TN/(TN+FP) # 1-specificity - false positive rate
    
    return tpr, fpr

def get_n_roc_coordinates(y_real, y_proba, n = 100):
    tpr_list = [0]
    fpr_list = [0]
    for i in range(n):
        threshold = i/n
        y_pred = y_proba[:, 1] > threshold
        tpr, fpr = calculate_tpr_fpr(y_real, y_pred)
        tpr_list.append(tpr)
        fpr_list.append(fpr)
    return tpr_list, fpr_list

def plot_roc_curve(tpr, fpr, ax, group, scatter = False):
    if scatter:
        sns.scatterplot(x = fpr, y = tpr)
    sns.lineplot(x = fpr, y = tpr, ax=ax,label=group)
    sns.lineplot(x = [0, 1], y = [0, 1], color = 'black', ax=ax, linestyle='--')
    plt.xlim(-0.05, 1.05)
    plt.ylim(-0.05, 1.05)
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")


_, axes = plt.subplots(1,3,figsize = (12, 4))
axes = axes.flatten()


for j, pf in enumerate(PROTECTED_FEATURES):
    proba_preds, true, protected_groups = y_proba_protected_groups(pf)
    for i in range(len(protected_groups)):
        tpr, fpr = get_n_roc_coordinates(true[i], proba_preds[i])
        plot_roc_curve(tpr, fpr, ax=axes[j], group=protected_groups[i],scatter = False)

axes[1].set_title("ROC Curve")
plt.legend()
plt.show()

In [ ]:
# Plot with sklearn
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay

fpr, tpr, _ = roc_curve(true[1], proba_preds[1][:,1])
roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr).plot()
roc_display.figure_.set_size_inches(5,5)
plt.plot([0, 1], [0, 1], color = 'g')


### Pareto from group example 2

In [ ]:
predictions = {}
result_metrics = {}
for l_value in tqdm(lambda_values):
    scaler = debias_data(protected_cols, nonprotected_cols, lambda_=l_value)
    kfold = KFold(n_splits=5, shuffle=True, random_state=1)

    model = LogisticRegression
    model_params = {"max_iter":500, "random_state":1}

    preds = test_model(model, X, y, kfold,scaler=scaler, model_params=model_params)
    predictions[f"{l_value:.2f}"] = preds
    
    acc_black = accuracy_score(data_y.HighCrime.values[IsBlack], preds[IsBlack])
    acc_non_black = accuracy_score(data_y.HighCrime.values[~IsBlack], preds[~IsBlack])
    recall_black, fpr_black = equalized_odds(data_y.HighCrime.values[IsBlack], preds[IsBlack])
    recall_non_black, fpr_non_black = equalized_odds(data_y.HighCrime.values[~IsBlack], preds[~IsBlack])
    num_high_black = np.sum(data_y.HighCrime.values[IsBlack]), np.sum(preds[IsBlack])
    num_highn_non_black = np.sum(data_y.HighCrime.values[~IsBlack]), np.sum(preds[~IsBlack])
    result_metrics[f"{l_value:.2f}"] = {"acc_black":acc_black,
                                        "acc_non_black": acc_non_black,
                                        "recall_black":recall_black, 
                                        "recall_non_black":recall_non_black,
                                         "fpr_black": fpr_black,
                                         "fpr_non_black": fpr_non_black,
                                        "num_high_black":num_high_black,
                                       "num_highn_non_black":num_highn_non_black}

In [ ]:
makro_acc = []
equal_odds_mse = []
for key, res in result_metrics.items():
    m_acc = (res["acc_black"] + res["acc_non_black"])/2
    se_recall = (res["recall_black"] - res["recall_non_black"])**2
    se_fpr = (res["fpr_black"] - res["fpr_non_black"])**2
    mse_total = (se_recall + se_fpr)/2
    
    makro_acc.append(m_acc)
    equal_odds_mse.append(mse_total)

In [ ]:
annotate_ax = [0, 7, 12, 18, 20, 23, 25, 29]
y_upper = [x + 10 for x in equal_odds_mse]

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,6))

ax.plot(makro_acc, equal_odds_mse, color = "black", linewidth = 0.5)

for j in np.arange(0,len(lambda_values),1):
    if j in annotate_ax:
        color = 'turquoise'
    else: 
        color = 'seagreen'
    ax.scatter(makro_acc[j], equal_odds_mse[j], color = color)

for i in annotate_ax:
    ax.annotate("$\lambda$ = "+str(np.round(lambda_values[i],2)), (makro_acc[i] + 0.01, equal_odds_mse[i]), fontsize = 12)

ax.fill_between(makro_acc, equal_odds_mse, y_upper, color = "seagreen", alpha = 0.1)

ax.set_xlim(right = 0.86)
ax.set_ylim(bottom = -0.005, top = 0.08)

plt.title("Logreg pareto front: fairness vs. accuracy", fontsize = 20)
plt.xlabel(r"Macro accuracy", fontsize = 14)
plt.ylabel("Mean squerred difference in equalized odds", fontsize = 14)
plt.show()